In [1]:
# 文件名称   ：roxml_to_dota.py
# 功能描述   ：把rolabelimg标注的xml文件转换成dota能识别的xml文件，
#             再转换成dota格式的txt文件
#            把旋转框 cx,cy,w,h,angle，或者矩形框cx,cy,w,h,转换成四点坐标x1,y1,x2,y2,x3,y3,x4,y4
import os
import xml.etree.ElementTree as ET
import math

cls_list=['0']
def edit_xml(xml_file, dotaxml_file):
    """
    修改xml文件
    :param xml_file:xml文件的路径
    :return:
    """
    tree = ET.parse(xml_file)
    objs = tree.findall('object')
    for ix, obj in enumerate(objs):
        x0 = ET.Element("x0")  # 创建节点
        y0 = ET.Element("y0")
        x1 = ET.Element("x1")
        y1 = ET.Element("y1")
        x2 = ET.Element("x2")
        y2 = ET.Element("y2")
        x3 = ET.Element("x3")
        y3 = ET.Element("y3")
        # obj_type = obj.find('bndbox')
        # type = obj_type.text
        # print(xml_file)
        if (obj.find('robndbox') == None):
            obj_bnd = obj.find('bndbox')
            obj_xmin = obj_bnd.find('xmin')
            obj_ymin = obj_bnd.find('ymin')
            obj_xmax = obj_bnd.find('xmax')
            obj_ymax = obj_bnd.find('ymax')
            #以防有负值坐标
            xmin = max(float(obj_xmin.text),0)
            ymin = max(float(obj_ymin.text),0)
            xmax = max(float(obj_xmax.text),0)
            ymax = max(float(obj_ymax.text),0)
            obj_bnd.remove(obj_xmin)  # 删除节点
            obj_bnd.remove(obj_ymin)
            obj_bnd.remove(obj_xmax)
            obj_bnd.remove(obj_ymax)
            x0.text = str(xmin)
            y0.text = str(ymax)
            x1.text = str(xmax)
            y1.text = str(ymax)
            x2.text = str(xmax)
            y2.text = str(ymin)
            x3.text = str(xmin)
            y3.text = str(ymin)
        else:
            obj_bnd = obj.find('robndbox')
            obj_bnd.tag = 'bndbox'  # 修改节点名
            obj_cx = obj_bnd.find('cx')
            obj_cy = obj_bnd.find('cy')
            obj_w = obj_bnd.find('w')
            obj_h = obj_bnd.find('h')
            obj_angle = obj_bnd.find('angle')
            cx = float(obj_cx.text)
            cy = float(obj_cy.text)
            w = float(obj_w.text)
            h = float(obj_h.text)
            angle = float(obj_angle.text)
            obj_bnd.remove(obj_cx)  # 删除节点
            obj_bnd.remove(obj_cy)
            obj_bnd.remove(obj_w)
            obj_bnd.remove(obj_h)
            obj_bnd.remove(obj_angle)

            x0.text, y0.text = rotatePoint(cx, cy, cx - w / 2, cy - h / 2, -angle)
            x1.text, y1.text = rotatePoint(cx, cy, cx + w / 2, cy - h / 2, -angle)
            x2.text, y2.text = rotatePoint(cx, cy, cx + w / 2, cy + h / 2, -angle)
            x3.text, y3.text = rotatePoint(cx, cy, cx - w / 2, cy + h / 2, -angle)


        # obj.remove(obj_type)  # 删除节点
        obj_bnd.append(x0)  # 新增节点
        obj_bnd.append(y0)
        obj_bnd.append(x1)
        obj_bnd.append(y1)
        obj_bnd.append(x2)
        obj_bnd.append(y2)
        obj_bnd.append(x3)
        obj_bnd.append(y3)

        tree.write(dotaxml_file, method='xml', encoding='utf-8')  # 更新xml文件


# 转换成四点坐标
def rotatePoint(xc, yc, xp, yp, theta):
    xoff = xp - xc;
    yoff = yp - yc;
    cosTheta = math.cos(theta)
    sinTheta = math.sin(theta)
    pResx = cosTheta * xoff + sinTheta * yoff
    pResy = - sinTheta * xoff + cosTheta * yoff
    return str(int(xc + pResx)), str(int(yc + pResy))


def totxt(xml_path, out_path):
    # 想要生成的txt文件保存的路径，这里可以自己修改

    files = os.listdir(xml_path)
    i=0
    for file in files:

        tree = ET.parse(xml_path + os.sep + file)
        root = tree.getroot()

        name = file.split('.')[0]

        output = out_path +'\\'+name + '.txt'
        file = open(output, 'w')
        i=i+1
        objs = tree.findall('object')
        for obj in objs:
            cls = obj.find('name').text
            box = obj.find('bndbox')
            x0 = int(float(box.find('x0').text))
            y0 = int(float(box.find('y0').text))
            x1 = int(float(box.find('x1').text))
            y1 = int(float(box.find('y1').text))
            x2 = int(float(box.find('x2').text))
            y2 = int(float(box.find('y2').text))
            x3 = int(float(box.find('x3').text))
            y3 = int(float(box.find('y3').text))
            if x0<0:
                x0=0
            if x1<0:
                x1=0
            if x2<0:
                x2=0
            if x3<0:
                x3=0
            if y0<0:
                y0=0
            if y1<0:
                y1=0
            if y2<0:
                y2=0
            if y3<0:
                y3=0
            for cls_index,cls_name in enumerate(cls_list):
                if cls==cls_name:
                    file.write("{} {} {} {} {} {} {} {} {} {}\n".format(x0, y0, x1, y1, x2, y2, x3, y3, cls,cls_index))
        file.close()
        # print(output)
        print(i)

if __name__ == '__main__':
    # -----**** 第一步：把xml文件统一转换成旋转框的xml文件 ****-----
    roxml_path = "detect_data/label"  
    dotaxml_path = 'detect_data/labelDota'  #
    out_path = 'detect_data/labelTxt'   
    filelist = os.listdir(roxml_path)
    for file in filelist:
        edit_xml(os.path.join(roxml_path, file), os.path.join(dotaxml_path, file))

    # -----**** 第二步：把旋转框xml文件转换成txt格式 ****-----
    totxt(dotaxml_path, out_path)



1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [2]:
import os
import shutil

def copy_and_rename(source_folder, destination_folder):
    # 创建目标文件夹
    destination_image_folder = os.path.join(destination_folder, 'images')
    destination_label_folder = os.path.join(destination_folder, 'label')
    os.makedirs(destination_image_folder, exist_ok=True)
    os.makedirs(destination_label_folder, exist_ok=True)

    # 初始化计数器
    count = 1

    # 遍历源文件夹中的每个子文件夹
    for subdir_name in os.listdir(source_folder):
        subdir_path = os.path.join(source_folder, subdir_name)
        
        # 确保是文件夹
        if os.path.isdir(subdir_path):
            # print(subdir_path)
            # # 遍历子文件夹中的每个文件
            for file_name in os.listdir(subdir_path):
                file_path = os.path.join(subdir_path, file_name)
                
                # 复制图片和对应的同名 XML 文件到目标文件夹
                if file_name.lower().endswith('.jpg'):
                    # 寻找同名的 XML 文件
                    xml_file_name = f"{os.path.splitext(file_name)[0]}.xml"
                    xml_file_path = os.path.join(subdir_path, xml_file_name)
                    
                    # 如果找到对应的 XML 文件，则复制到目标文件夹并改名
                    if os.path.exists(xml_file_path):
                        # 构造目标文件名
                        new_image_name = f"{count}.jpg"
                        new_xml_name = f"{count}.xml"
                        #  复制图片,xml并改名
                        shutil.copy(file_path, os.path.join(destination_image_folder, new_image_name))
                        shutil.copy(xml_file_path, os.path.join(destination_label_folder, new_xml_name))
                        print(os.path.join(destination_image_folder, new_image_name))
                        print(os.path.join(destination_label_folder, new_xml_name))
                        # 更新计数器
                        print(count)
                        count += 1
        # else:
        #     print('*********')

if __name__ == "__main__":
    source_folder = "eyedata"
    destination_folder = "detect_data"
    
    copy_and_rename(source_folder, destination_folder)


detect_data/images/1.jpg
detect_data/label/1.xml
1
detect_data/images/2.jpg
detect_data/label/2.xml
2
detect_data/images/3.jpg
detect_data/label/3.xml
3
detect_data/images/4.jpg
detect_data/label/4.xml
4
detect_data/images/5.jpg
detect_data/label/5.xml
5
detect_data/images/6.jpg
detect_data/label/6.xml
6
detect_data/images/7.jpg
detect_data/label/7.xml
7
detect_data/images/8.jpg
detect_data/label/8.xml
8
detect_data/images/9.jpg
detect_data/label/9.xml
9
detect_data/images/10.jpg
detect_data/label/10.xml
10
detect_data/images/11.jpg
detect_data/label/11.xml
11
detect_data/images/12.jpg
detect_data/label/12.xml
12
detect_data/images/13.jpg
detect_data/label/13.xml
13
detect_data/images/14.jpg
detect_data/label/14.xml
14
detect_data/images/15.jpg
detect_data/label/15.xml
15
detect_data/images/16.jpg
detect_data/label/16.xml
16
detect_data/images/17.jpg
detect_data/label/17.xml
17
detect_data/images/18.jpg
detect_data/label/18.xml
18
detect_data/images/19.jpg
detect_data/label/19.xml
19
d

KeyboardInterrupt: 

In [1]:
import torch

# 查看PyTorch版本
print(f"PyTorch 版本: {torch.__version__}")

# 检查CUDA是否可用
cuda_available = torch.cuda.is_available()

if cuda_available:
    # 查看当前CUDA设备数量
    cuda_device_count = torch.cuda.device_count()
    print(f"CUDA 可用，设备数量: {cuda_device_count}")

    # 查看当前CUDA设备索引
    current_cuda_device = torch.cuda.current_device()
    print(f"当前CUDA设备索引: {current_cuda_device}")

    # 查看当前CUDA设备的名称
    current_cuda_device_name = torch.cuda.get_device_name(current_cuda_device)
    print(f"当前CUDA设备名称: {current_cuda_device_name}")

    # 查看CUDA版本
    cuda_version = torch.version.cuda
    print(f"CUDA 版本: {cuda_version}")
else:
    print("CUDA 不可用")

PyTorch 版本: 1.10.1+cu113
CUDA 可用，设备数量: 4
当前CUDA设备索引: 0
当前CUDA设备名称: NVIDIA GeForce RTX 3090
CUDA 版本: 11.3


In [1]:
import os

def save_image_paths_to_txt(folder_path, txt_file_path):
    # 获取文件夹中所有图片文件的完整路径
    image_paths = [os.path.join(folder_path, file) 
                   for file in os.listdir(folder_path) if file.endswith(('.jpg', '.png', '.jpeg'))]

    # 写入完整路径到 txt 文件
    with open(txt_file_path, 'w') as txt_file:
        txt_file.write('\n'.join(image_paths))

# 替换为实际的文件夹路径和输出txt文件路径
folder_path = "detect_data/images/val"
txt_file_path = "detect_data/val.txt"

# 调用函数保存相对路径到txt文件
save_image_paths_to_txt(folder_path, txt_file_path)
